In [1]:
p_storage_dir = "../../storage"
p_notebooks_root = None

In [2]:
if p_notebooks_root is not None:
    import os
    os.chdir(p_notebooks_root)

In [3]:
import ast
import unicodedata
import re

import pandas as pd
from pathlib import Path

IFG_PRODUZ_STORAGE_DIR = Path(f"{p_storage_dir}/ifg_produz")

## Currículos -> Cidades

Notebook focado na ligação de cada currículo com a sua devida cidade que já foi previamente pré-processada.

In [4]:
curriculos = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "intermediate/curriculos.csv", delimiter=";")

curriculos.sample(n=5)

,codigo,nome_completo,nome_completo_sem_acento,link,nome_citacoes_bibliograficas,uf_nascimento,cidade_nascimento,resumo,palavra_chave,campus_atual,categoria,aceitando_email,siape,email,data_nascimento,sigla_campus_atual
1610,177,Cloves da Silva Junior,Cloves da Silva Junior,http://lattes.cnpq.br/0957588745949116,"SILVA JUNIOR, C.;SILVA JUNIOR, Cloves.;JUNIOR,...",GO,Goiás,Aluno regular do curso de Doutorado do Program...,"[{'palavra': 'romances', 'correta': 'Romances'...",CÂMPUS ITUMBIARA,Docente,True,3007748,cloves.junior@ifg.edu.br,1989-01-05,ITU
1745,878,Paulo César Faria Rodrigues,Paulo Cesar Faria Rodrigues,http://lattes.cnpq.br/4628092481645526,"RODRIGUES, P. C. F.",NaN,NaN,Possui graduação em Administração pelo Centro ...,NaN,CÂMPUS LUZIÂNIA,TAE,True,1829383,paulo.rodrigues@ifg.edu.br,1978-03-18,LUZ
1650,294,Thiago VedoVatto,Thiago VedoVatto,http://lattes.cnpq.br/1555798555635250,"VEDOVATTO, T.",GO,Jataí,Possui Licenciatura Plena em Matemática pela U...,"[{'palavra': 'matematica', 'correta': 'Matemát...",CÂMPUS GOIÂNIA,Docente,True,1768136,thiago.vedovatto@ifg.edu.br,1986-02-13,GYN
482,709,Renata Luiza da Costa,Renata Luiza da Costa,http://lattes.cnpq.br/3773019506029897,"COSTA, R. L.;COSTA, RENATA LUIZA DA;da Costa, ...",GO,Goiânia,Doutora em Edudação pela PUC-Goiás (2015). Dou...,"[{'palavra': 'educacao', 'correta': 'Educação'...",CÂMPUS INHUMAS,Docente,True,1554456,renata.costa@ifg.edu.br,1979-06-22,INH
208,318,Aline Seixas Ferro,Aline Seixas Ferro,http://lattes.cnpq.br/1705903018416588,"FERRO, A. S.",NaN,NaN,"Psicóloga do Instituto Federal de Educação, Ci...",NaN,CÂMPUS GOIÂNIA OESTE,TAE,True,1972770,aline.ferro@ifg.edu.br,1988-09-09,OES


In [5]:
cidades = pd.read_csv(IFG_PRODUZ_STORAGE_DIR / "preprocessed/cidades.csv", delimiter=";", converters={'aliases': ast.literal_eval})

cidades.sample(n=5)

,nome,nome_ascii,sigla_uf,aliases,codigo
151,Caçapava,cacapava,SP,[Caçapava],3ca5738f-7c03-46cf-b6d6-be3847a7b7d8
282,Taguatinga,taguatinga,DF,"[Taguatinga, Taguatinga Sul]",e2878ea4-d806-4348-a685-eb2001487d04
77,São Francisco de Goiás,sao francisco de goias,GO,[São Francisco de Goiás],bc5a5c85-6930-427d-b875-f25845040f2f
233,Palmas,palmas,TO,[Palmas],099a5b72-d281-48c2-9d9b-5d70c8fba4a5
184,Santos,santos,SP,[Santos],30d3d36a-11aa-4cac-a33c-965ccaa30117


### Ligar cada currículo com sua devida cidade de nascimento

In [6]:
def normalizar_string(s):
    if pd.isna(s):
        return
    
    # Transforma todos os caracteres em minúsculas
    s = s.lower()
    
    # Remove acentos das letras
    s = unicodedata.normalize("NFD", s).encode("ascii", "ignore").decode("utf-8")

    # Substitui todos os caracteres que não são letras por um espaço em branco
    s = re.sub(r'[^A-Za-z]', ' ', s)

    # Substitui espaços consecutivos por apenas um espaço
    s = re.sub(r'\s{2,}', ' ', s)

    # Remove os espaços em branco do começo e fim
    s = s.strip()

    return s

In [7]:
curriculos['cidade_nascimento_normalizada'] = curriculos.loc[:, 'cidade_nascimento'].apply(normalizar_string)

In [8]:
d = curriculos.loc[1490, ['siape', 'cidade_nascimento', 'cidade_nascimento_normalizada', 'uf_nascimento']]

for _, cidade in cidades.iterrows():
    if cidade['sigla_uf'] == d['uf_nascimento'] and (d['cidade_nascimento_normalizada'] == cidade['nome_ascii'] 
                                                     or d['cidade_nascimento'] in cidade['aliases']):
        d['codigo_cidade_nascimento'] = cidade['codigo']
        break

d

siape                                                         1288789
cidade_nascimento                                            Brasília
cidade_nascimento_normalizada                                brasilia
uf_nascimento                                                      DF
codigo_cidade_nascimento         e1f1093c-df27-425b-bf91-d3892f0cb428
Name: 1490, dtype: object

In [9]:
def link_curriculo_to_cidade(row, cidades):
    for _, cidade in cidades.iterrows():
        if row['cidade_nascimento_normalizada'] == cidade['nome_ascii'] or row['cidade_nascimento'] in cidade['aliases']:
            row['codigo_cidade_nascimento'] = cidade['codigo']
            break

    return row

In [10]:
curriculos = curriculos.apply(link_curriculo_to_cidade, args=(cidades,), axis=1)

In [11]:
curriculos = curriculos.drop(columns=['cidade_nascimento_normalizada'])

In [137]:
curriculos.sample(n=10)

,aceitando_email,campus_atual,categoria,cidade_nascimento,codigo,codigo_cidade_nascimento,data_nascimento,email,link,nome_citacoes_bibliograficas,nome_completo,nome_completo_sem_acento,palavra_chave,resumo,siape,sigla_campus_atual,uf_nascimento
1241,True,CÂMPUS FORMOSA,Docente,NATAL,651,a952b204-a839-45f1-81eb-f20ecddba97c,1988-03-25,andre.dantas@ifg.edu.br,http://lattes.cnpq.br/3443223409799104,"DANTAS, A. A. N.;DANTAS, ANDRÉ AUGUSTO NÓBREGA",Andre Augusto Nobrega Dantas,Andre Augusto Nobrega Dantas,"[{'palavra': 'estudo', 'correta': 'Estudo', 'i...",Possui graduação em Engenharia Civil pela Univ...,1230337,FOR,RN
959,True,CÂMPUS GOIÂNIA,Docente,Araçatuba,1447,24924ad9-8551-4acf-9ea6-ad072f2b68d0,1988-05-12,edipo.cremon@ifg.edu.br,http://lattes.cnpq.br/7888258901938956,"CREMON, É. H.;Cremon, Édipo Henrique;CREMON, E...",Édipo Henrique Cremon,Edipo Henrique Cremon,"[{'palavra': 'imagens', 'correta': 'Imagens', ...",Geógrafo pela Universidade Estadual de Maringá...,2285085,GYN,SP
888,True,CÂMPUS GOIÂNIA,Docente,Goiania,1338,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1980-02-29,limerce.lopes@ifg.edu.br,http://lattes.cnpq.br/7266113954157102,"LOPES, L. F.",Limerce Ferreira Lopes,Limerce Ferreira Lopes,"[{'palavra': 'letramento', 'correta': 'Letrame...",Graduada em Letras com habilitação em Portuguê...,1796028,GYN,GO
609,True,REITORIA,TAE,Goiânia,895,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1970-09-25,kenia.hidalgo@ifg.edu.br,http://lattes.cnpq.br/4708038363452351,"SILVA, K. R.;HIDALGO, K. R. da S.;HIDALGO, K. ...",Kenia Ribeiro da Silva Hidalgo,Kenia Ribeiro da Silva Hidalgo,"[{'palavra': 'aprendizagem', 'correta': 'Apren...",Doutora em Educação pela PUC-GO. Mestra em Edu...,3092355,REI,GO
305,True,CÂMPUS GOIÂNIA,TAE,GOIÂNIA,458,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1985-01-23,camila.franco@ifg.edu.br,http://lattes.cnpq.br/2398750759537246,"FRANCO, C. N.",Camila Noleto Franco,Camila Noleto Franco,NaN,Possui graduação em Direito pela Pontifícia Un...,1649936,GYN,GO
1800,True,CÂMPUS ANÁPOLIS,TAE,Goiânia,840,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1985-09-27,tales.tavares@ifg.edu.br,http://lattes.cnpq.br/4433615594721099,"TAVARES, T. O.",Tales de Oliveira Tavares,Tales de Oliveira Tavares,"[{'palavra': 'biofisica', 'correta': 'Biofísic...",Graduado no curso de Biologia com outorga de B...,1006480,ANA,GO
937,True,CÂMPUS CIDADE DE GOIÁS,Docente,Gurupi,1431,e224dd8f-9769-44b2-8509-f3c7c6f2a242,1981-11-11,alemar.sousa@ifg.edu.br,http://lattes.cnpq.br/7771133603438281,"SOUSA, Alemar Moreira de",Alemar Moreira de Sousa,Alemar Moreira de Sousa,NaN,possui graduação em Geografia pela Universidad...,2573433,GOI,TO
355,True,CÂMPUS ÁGUAS LINDAS,Docente,Rio Branco,533,9098d857-08f5-46ca-88d9-7c779ad9d65e,1981-07-19,rossemildo.santos@ifg.edu.br,http://lattes.cnpq.br/2886499316177159,"SANTOS, R. S.",Rossemildo da Silva Santos,Rossemildo da Silva Santos,NaN,Licenciatura em Letras Português-Espanhol e su...,1569507,LIN,AC
1708,True,CÂMPUS APARECIDA DE GOIÂNIA,TAE,Goiânia,513,8648f216-75f9-44b6-a00c-7d8f5a646c2d,1983-08-16,daniela.rezende@ifg.edu.br,http://lattes.cnpq.br/2725347320666971,"REZENDE, D. R.;REZENDE, DANIELA R.;REZENDE, D.R.",Daniela Rodrigues de Rezende,Daniela Rodrigues de Rezende,NaN,Daniela Rodrigues de Rezende é Técnica em Sane...,1941927,APA,GO
630,True,CÂMPUS JATAÍ,TAE,Jataí,925,a45d3350-91da-4db0-9883-104c996a85e1,1972-09-30,neiva.freitas@ifg.edu.br,http://lattes.cnpq.br/4914615367829274,"FREITAS, N. S.",Neiva Santos de Freitas,Neiva Santos de Freitas,NaN,Atualmente é Assistente social na Universidade...,1411262,JAT,GO


In [12]:
curriculos.isna().sum()

aceitando_email                   0
campus_atual                      0
categoria                         0
cidade_nascimento               238
codigo                            0
codigo_cidade_nascimento        238
data_nascimento                   8
email                             0
link                             18
nome_citacoes_bibliograficas      0
nome_completo                     0
nome_completo_sem_acento          0
palavra_chave                   725
resumo                          167
siape                             0
sigla_campus_atual                0
uf_nascimento                   237
dtype: int64

### Exportar

In [138]:
export_path = Path(IFG_PRODUZ_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

curriculos.to_csv(export_path / "curriculos.csv", sep=";", index=False)